In [17]:
from google.colab import drive
import json
import polars as pl
import pandas as pd
import re
from collections import Counter

drive.mount('/content/drive')
ubicaciones = ['review-Florida_parquet', 'review-New_York_parquet']
carpetas = {'review-Florida_parquet': 19, 'review-New_York_parquet': 18}
palabras_a_buscar = ['wifi', 'gym', 'restaurant', 'piscina', 'laundry', 'kindergarten', 'pool', 'parking lot',
                     'massage', 'casino', 'resort', 'fitness', 'tennis', 'taxi', 'Swimming', 'food', 'parking', 'central heating', 'central heat']

def leer_y_procesar_parquet(ubicaciones, carpetas):
    dfs = []

    # Función para leer y convertir un archivo Parquet
    def leer_y_convertir_parquet(n, ubicacion):
        file_path = f'/content/drive/MyDrive/reviews/{ubicacion}/{n}.parquet'
        df = pl.read_parquet(file_path)
        df_pandas = df.to_pandas()
        return df_pandas

    # Leer y procesar los archivos Parquet
    for ubicacion in ubicaciones:
        for i in range(1, carpetas[ubicacion] + 1):
            df = leer_y_convertir_parquet(str(i), ubicacion)
            dfs.append(df)

    # Concatenar todos los DataFrames en uno solo
    df1 = pd.concat(dfs)
    df1 = df1[['text', 'gmap_id']]
    ruta2 = '/content/drive/MyDrive/reviews/hoteles.csv'
    df2 = pd.read_csv(ruta2)
    df2 = df2[['name', 'gmap_id', 'City', 'State', 'url']]
    df = pd.merge(df2, df1, on='gmap_id', how='left')
    df = df.dropna(subset=['text'])
    df['amenidades'] = ''
    return df


def extraer_amenidades(df, palabras_a_buscar):
    # Iterar sobre cada fila del DataFrame
    for index, row in df.iterrows():
        texto = row['text']

        # Verificar si el texto no es None
        if texto is not None:
            # Inicializar una lista vacía para almacenar las amenidades encontradas
            amenidades_encontradas = []

            # Buscar palabras clave de amenidades en el texto y agregarlas a la lista de amenidades encontradas
            for palabra in palabras_a_buscar:
                if palabra in texto:
                    amenidades_encontradas.append(palabra)

            # Asignar la lista de amenidades encontradas a la columna 'amenidades' para esta fila
            df.at[index, 'amenidades'] = amenidades_encontradas
        else:
            # Si el texto es None, asignar una lista vacía a 'amenidades'
            df.at[index, 'amenidades'] = []

    # Eliminar filas duplicadas basadas en el gmap_id
    df = df.drop_duplicates(subset=['gmap_id'])

    return df

df_final = leer_y_procesar_parquet(ubicaciones, carpetas)
final=extraer_amenidades(df_final, palabras_a_buscar)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
